In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, AdamW, get_scheduler, AutoModel

In [ ]:
import os

# Define the new directory path
new_directory = '/content/drive/MyDrive/Colab Notebooks'

# Change the current working directory to the new one
os.chdir(new_directory)

# Verify the change by printing the current working directory
print("Current working directory:", os.getcwd())

Current working directory: /content/drive/MyDrive/Colab Notebooks


In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('Datasets/data_amazon_product_reviews_video_games.csv')

In [ ]:
df.drop(labels= ['Unnamed: 0', 'reviewerID', 'asin', 'reviewerName', 'helpful',
       'unixReviewTime', 'reviewTime'], axis= 1, inplace= True)

In [ ]:
df.dropna(inplace= True)
#df.isna().sum()

In [ ]:
df['overall']= df['overall'].astype(dtype= 'int64')
#df.info()

In [ ]:
# Assuming your CSV file has two columns: 'text' for the input text and 'label' for the class label
df['new_text']= df['reviewText'] + ' ' + df['summary']

texts= df['new_text'].tolist()
labels= df['overall'].tolist()

In [ ]:
# Define hyperparameters
learning_rate = 1e-3   # Learning rate or step size in optimizer
batch_size = 16    # Batch size of training / evaluation / test dataset
weight_decay = 1e-4   # Weight decay ratio in optimizer
dropout_rate = 0.1   # Dropout ratio of the additional hidden layer
num_epochs = 3   # Number of epocks
hidden_dim = 256   # 256 classes as an additional hidden layer
output_dim = 5    # 5 classes for sentiment analysis

In [ ]:
# Split the data into train and test sets (80% train, 20% test)
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Further split the test set into dev and test sets (50% dev, 50% test)
dev_texts, test_texts, dev_labels, test_labels = train_test_split(test_texts, test_labels, test_size=0.5, random_state=42)

In [ ]:
# Load the pre-trained BERT tokenizer and model
checkpoint= "LiYuan/amazon-review-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
# Tokenize the input texts
tokenized_train_texts = tokenizer(train_texts, padding=True, truncation=True, return_tensors='pt')

tokenized_dev_texts = tokenizer(dev_texts, padding=True, truncation=True, return_tensors='pt')

tokenized_test_texts = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')

In [ ]:
# Convert the labels to tensor
train_labels = torch.tensor(train_labels)

dev_labels = torch.tensor(dev_labels)

test_labels = torch.tensor(test_labels)

In [ ]:
# Create TensorDatasets and DataLoaders for train, dev, and test sets
train_dataset = TensorDataset(tokenized_train_texts['input_ids'], tokenized_train_texts['attention_mask'], train_labels)

dev_dataset = TensorDataset(tokenized_dev_texts['input_ids'], tokenized_dev_texts['attention_mask'], dev_labels)

test_dataset = TensorDataset(tokenized_test_texts['input_ids'], tokenized_test_texts['attention_mask'], test_labels)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)

dev_dataloader = DataLoader(dev_dataset, batch_size= batch_size, shuffle= False)

test_dataloader = DataLoader(test_dataset, batch_size= batch_size, shuffle= False)

In [ ]:
# Define your model architecture
class SentimentClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, output_dim, dropout_rate):
        super(SentimentClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(bert_model.config.hidden_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim= 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids= input_ids, attention_mask= attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [ ]:
# LiYuan Model
bert_model = AutoModel.from_pretrained(checkpoint)

# Freeze the BERT model parameters
for param in bert_model.parameters():
    param.requires_grad = False

In [ ]:
# Create an instance of the SentimentClassifier
model = SentimentClassifier(bert_model, hidden_dim, output_dim, dropout_rate)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Define the optimizer for training the softmax layer
optimizer = optim.Adam(model.parameters(), lr= learning_rate, weight_decay= weight_decay)

In [ ]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer= optimizer,
    num_warmup_steps= 0,
    num_training_steps= num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

  0%|          | 0/7500 [00:00<?, ?it/s]

In [ ]:
# Train the model
best_dev_accuracy = 0.0
best_model_state_dict = None
Validation_results= []

for epoch in range(num_epochs):
    # Training loop
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, batch_labels = batch

        input_ids= input_ids.to(device)
        attention_mask= attention_mask.to(device)
        batch_labels= batch_labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids= input_ids, attention_mask= attention_mask)
        loss = criterion(outputs, batch_labels)  
        loss.backward()
        optimizer.step()
        ####
        lr_scheduler.step()
        progress_bar.update(1)


    # Validation loop
    model.eval()
    dev_correct = 0
    total_dev = 0
    y_true = []
    y_pred = []
    loss_epoch= []
    with torch.no_grad():
        for batch in dev_dataloader:
            input_ids, attention_mask, batch_labels = batch

            input_ids= input_ids.to(device)
            attention_mask= attention_mask.to(device)
            batch_labels= batch_labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            #logits = outputs.logits
            logits = outputs
            _, predicted = torch.max(logits, 1)

            # Append true labels and predicted labels for later use
            y_true.extend(batch_labels.tolist())
            y_pred.extend(predicted.tolist())

            # Calculate the loss
            loss = criterion(logits, batch_labels)
            loss_epoch.append(loss)

    # Calculate accuracy and F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate the average loss
    loss_epoch_np = [tensor.cpu().detach().numpy() for tensor in loss_epoch]
    average_loss= np.mean(loss_epoch_np)
    print(f'epoch No. : {epoch}, Devset Accuracy : {round(accuracy,5)}, Devset f1_score : {round(f1,5)}, Average loss: {round(average_loss.tolist(),5)}')

    Validation_results.append([accuracy, f1, average_loss])

    if accuracy > best_dev_accuracy:
        best_dev_accuracy = accuracy
        # Save the best model (optional)
        best_model_state_dict = model.state_dict()

epoch No. : 0, Devset Accuracy : 0.67894, Devset f1_score : 0.66724, Average loss: 0.78317
epoch No. : 1, Devset Accuracy : 0.68194, Devset f1_score : 0.67171, Average loss: 0.7734
epoch No. : 2, Devset Accuracy : 0.67814, Devset f1_score : 0.66723, Average loss: 0.76863


In [ ]:
# Saving Testset Results
data = {
    'Validation_results': Validation_results,
}
df = pd.DataFrame(data)
df.to_csv('Outputs/Main-10-LiYuan-Two layers-Not OPT_Validation_results.csv', index= False)

In [ ]:
# Load the best model state dict
if best_model_state_dict is not None:
    model.load_state_dict(best_model_state_dict)

    # Define the directory path to save the model
    save_path = 'Saved Models/Main-10-LiYuan-Two layers-Not OPT.pth'  

    # Save the model state dictionary and other relevant information
    torch.save({
        'model_state_dict': best_model_state_dict,
        'tokenizer': tokenizer  
    }, save_path)

In [ ]:
# Evaluate on the test set
model.eval()
y_true_test = []
y_pred_test = []
loss_epoch= []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, batch_labels = batch

        input_ids= input_ids.to(device)
        attention_mask= attention_mask.to(device)
        batch_labels= batch_labels.to(device)

        outputs = model(input_ids= input_ids, attention_mask= attention_mask)
        logits = outputs
        _, predicted = torch.max(logits, 1)

        # Append true labels and predicted labels for later use
        y_true_test.extend(batch_labels.tolist())
        y_pred_test.extend(predicted.tolist())

        # Calculate the loss
        loss = criterion(logits, batch_labels)
        loss_epoch.append(loss)

# Calculate accuracy and F1 score for the test set
test_accuracy = accuracy_score(y_true_test, y_pred_test)
test_f1 = f1_score(y_true_test, y_pred_test, average='weighted')

# Calculate the average loss
loss_epoch_np = [tensor.cpu().detach().numpy() for tensor in loss_epoch]
average_loss= np.mean(loss_epoch_np)

print(f"Testset accuracy: {round(test_accuracy,5)} , Testset F1 score: {round(test_f1,5)}, Average loss: {round(average_loss.tolist(),5)}")
Test_results= [test_accuracy, test_f1, average_loss]

Testset accuracy: 0.69294 , Testset F1 score: 0.68326, Average loss: 0.74917


In [ ]:
# Saving Testset Results
data = {
    'Test_results': Test_results
}
df = pd.DataFrame(data)
df.to_csv('Outputs/Main-10-LiYuan-Two layers-Not OPT_Test_results.csv', index= False)